## This set of codes is designed to help you run a velmex stage. 

It builds heavily on codes by Steve Brown, and those available here: 
https://github.com/unkatoco/Velmex/blob/master/velmex.py

Our stage has 2 axis, one horizontal (x) and one vertical (y). 

There are codes to: 
1. Find the serial ports on your machine.
2. Open communication and check that you are indeed communicating with the stage.
3. Set the zero location of the stage.
4. Move the stage a certain number of rotations.
5. Move the stage a certain distance.
6. Get the stage's location.
7. Move the stage to a specific position.

In [1]:
import numpy as np
import sys
import time
import serial
ser = serial.Serial()

First find the serial ports.  

In [2]:
import serial.tools.list_ports; 
print([comport.device for comport in serial.tools.list_ports.comports()])

['/dev/cu.BLTH', '/dev/cu.Bluetooth-Incoming-Port', '/dev/cu.usbserial-AL030TI8']


Now setup the parameters for your connection.

In [16]:
ser.port="/dev/tty.usbserial-AL030TI8"  #change this to the path to your serial port (or possibly just its name depending on your OS)

ser.baudrate = 9600
ser.bytesize = serial.EIGHTBITS		#number of bits per bytes
ser.parity = serial.PARITY_NONE		#set parity check: no parity
ser.stopbits = serial.STOPBITS_ONE	#number of stop bits

#ser.timeout = None		#block read
ser.timeout = 1			#non-block read
#ser.timeout = 2		        #timeout block read

ser.xonxoff = False		#disable software flow control
ser.rtscts = False		#disable hardware (RTS/CTS) flow control
ser.dsrdtr = False		#disable hardware (DSR/DTR) flow control
ser.writeTimeout = 2	        #timeout for write

ser.open() 

Now get a status report.  

This should return R, but J is OK too, ignore the '^'s.

In [18]:
 #connect to the stage
ser.write("V".encode())  #check that you are connected, 
out=ser.readline()
print(out.decode())

R


Now set the zero location of the stage.  This sets it to bottom left, but there are comments showing how to put it elsewhere.

In [19]:
ser.write("F,C,I1M-0,R".encode())  #move all the way to the left (-), change the number to choose a new position
time.sleep(30)
ser.write("N".encode())
time.sleep(.5)
ser.write("F,C,I2M0,R".encode())  #move all the way to the bottom (+)
time.sleep(30)  #wait 30 seconds, note if you don't wait long enough this will not work
ser.write("N".encode())  #reset the counter to zero here 




1

Get the stage's current location

In [6]:
def GetLoc():
    ser.write("Y".encode())  #tell the stage you want to know about the y (vertical) axis
    pos=ser.readline()       #read the info from the scope
    pos1=pos.decode()        #change it from bytes to numbers

    #remove weird characters from the beginning
    ii=0
    while not(pos1[ii]=='+' or pos1[ii]=='-'):
        ii=ii+1
    # extract sign
    sign=1                   
    if pos1[ii]=='-':
        sign=-1
    #convert to integer
    pos1 = int(''.join([x for x in pos1[ii:] if x.isdigit()])) # parseint
    #incorporate the sign into the integer
    pos1=sign*pos1

    #Now repeat for the x-axis
    ser.write("X".encode())
    pos=ser.readline()
    pos2=pos.decode()
    ii=0
    while not(pos2[ii]=='+' or pos2[ii]=='-'):
        ii=ii+1
    sign=1
    if pos2[ii]=='-':
        sign=-1
    pos2 = int(''.join([x for x in pos2[ii:] if x.isdigit()])) # parseint
    pos2=sign*pos2
            #Y, X
    return pos1,pos2


In [7]:
posy,posx=GetLoc()
print(posx,posy)

0 0


Move the stage a certain distance (in mm).  Note that this incorporates a check that you will not move past the boundary of the stage's range (with the laser attached), but that check is not perfect, and can be messed up if you have moved the stage a lot.  You should reset the origin frequently if you are using this code to move large distances.  Note also that there are several pauses in this code to wait for the stage to move.  You can change this, but if you make it too short then it will not move the second axis.

In [8]:
def Move(Xmm,Ymm,waittime=10,Xmax=43000,Ymin=-48000):
    #Move Xmm in the x-direction (- numbers goes to left, + numbers goes to right)
    #Move Ymm in the y-direction (- numbers goes up, + numbers goes to down)
    #waittime=10 time to wait between motions for the stage to finish moving
    #Xmax=43000 maximum distance stage can move in X (in revolutions)
    #Ymax=-48000 maximum distance stage can move in Y (in revolutions)
    #With failsafe to ensure that it doesn't go too far on any axis.
    #But that failsafe is not perfect, you should reset the origin before using this code for large motions
   
    #First convert the distances to revolutions:
    StepsPermm = 400 / (2.54)   #stage motion is 400 revolutions per 0.1 inch, convert to mm
    
    Distx_rev=Xmm*StepsPermm
    Disty_rev=Ymm*StepsPermm
   
    #Find current position (for error trapping)
    posy,posx=GetLoc()
    #wait for it to finish
    time.sleep(waittime)
    #Move the stage in x if distance is OK
    if ((posx+Distx_rev)<Xmax and (posx+Distx_rev)>0):
        ser.write(("F,C,I1M"+str(round(Distx_rev,0))+",R").encode()) 
    else: 
        print("Asked to move too far in X, moving to max allowed")
        if (posx+Distx_rev)>=Xmax:
            ser.write(("F,C,IA1M"+str(round(Xmax,0))+",R").encode())
        else:
            ser.write(("F,C,IA1M"+str(0)+",R").encode())
     
    #Wait for it to finish moving
    time.sleep(waittime)
    
    #repeat in Y
    if ((posy+Disty_rev)>Ymin and (posy+Disty_rev)<0):
        ser.write(("F,C,I2M"+str(round(Disty_rev,0))+",R").encode())  
    else: 
        print("Asked to move too far in Y, moving to max allowed")
        if (posy+Disty_rev)<Ymin:
            ser.write(("F,C,IA2M"+str(Ymin)+",R").encode())
        else:
            ser.write(("F,C,IA2M"+str(0)+",R").encode())




Move the stage to a specific location in mm from the origin.  Similar to Move, but instead of moving a certain distance, it moves the stage to a specific location.  Same warnings apply about making sure that the origin is set properly, particularly if you will be anywhere near the limits of the stage's motion.

In [9]:
def CoordMove(Xmm,Ymm,waittime=10,Xmax=43000,Ymin=-48000):
    #Move Xmm in the x-direction (- numbers goes to left, + numbers goes to right)
    #Move Ymm in the y-direction (- numbers goes up, + numbers goes to down)
    #waittime=10 time to wait between motions for the stage to finish moving
    #Xmax=43000 maximum distance stage can move in X (in revolutions)
    #Ymax=-48000 maximum distance stage can move in Y (in revolutions)

#With failsafe to ensure that it doesn't go too far on any axis.
    
    #First convert the distances to revolutions:
    StepsPermm = 400 / (2.54)   #stage motion is 400 revolutions per 0.1 inch, convert to mm
    
    Distx_rev=Xmm*StepsPermm
    Disty_rev=Ymm*StepsPermm
   
    #Find current position (for error trapping)
    posy,posx=GetLoc()
    #wait for it to finish
    time.sleep(waittime)
    #Move the stage in x if distance is OK
    if ((Distx_rev)<Xmax and Distx_rev>0):
        ser.write(("F,C,IA1M"+str(round(Distx_rev,0))+",R").encode()) 
    else: 
        print("Asked to move too far in X, moving to max location")
        if Distx_rev>Xmax:
            ser.write(("F,C,IA1M"+str(round(Xmax,0))+",R").encode())
        else: 
            ser.write(("F,C,IA1M"+str(0)+",R").encode())
     
    #Wait for it to finish moving
    time.sleep(waittime)
    
    #repeat in Y
    if ((Disty_rev)>Ymin and Disty_rev<0):
        ser.write(("F,C,I2M"+str(round(Disty_rev,0))+",R").encode())  
    else: 
        print("Asked to move too far in Y, moving to max allowed")
        if Disty_rev<0:
            ser.write(("F,C,IA2M"+str(Ymin)+",R").encode())
        else:
            ser.write(("F,C,IA2M"+str(0)+",R").encode())

                      



Demo of how to use the move function

In [20]:
Move(10,-10)

Change the wait-time because it's a short move

In [21]:
Move(3,-3,waittime=3)

Demo of the coordinate move, which is quite similar!

In [22]:
CoordMove(20,-20)

Clean-up, close the connection to the device when you are finished.

In [13]:
ser.close()

Use this if the motor is spinning and the stage is at its limits (or press the red button on the controller).

In [14]:
ser.write("K".encode())  #Emergency stop, all axes

PortNotOpenError: Attempting to use a port that is not open

In [ ]:
ser.flushInput()	        #flush input buffer, discarding all its contents
ser.flushOutput()